---
title: "Trade Envy Report"
author: "Thomas Niedermayer"
date: last-modified
engine: jupyter
execute:
  echo: false
  enabled: true
---

In [1]:
import pandas as pd
from cow_amm_trade_envy.models import Contracts
from cow_amm_trade_envy.datasources import get_logs


def analyze_surplus(file_path, pool_address):
    pool_address = pool_address.lower()

    def isin_pool(topic):
        assert "0x" == pool_address[:2]
        return pool_address[2:] in topic

    def log_is_used(log):
        return any(isin_pool(topic) for topic in log["topics"])

    def logs_are_used(logs):
        return any(log_is_used(log) for log in logs)

    file = pd.read_csv(file_path)

    file["logs"] = file["call_tx_hash"].apply(get_logs)
    file["is_used"] = file["logs"].apply(logs_are_used)

    surplus_unused = file[~file["is_used"]]["trade_envy"].sum()
    surplus_used = file[file["is_used"]]["trade_envy"].sum()

    return {
        "total_unused_surplus": surplus_unused / 10**18,
        "total_used_surplus": surplus_used / 10**18,
    }

In [2]:
file_path = "data/cow_amm_missed_surplus.csv"
pool_address = Contracts.USDC_WETH_POOL

surplus_results = analyze_surplus(file_path, pool_address)
print(f"Total unused surplus: {surplus_results['total_unused_surplus']}")
print(f"Total used surplus: {surplus_results['total_used_surplus']}")

Total unused surplus: -0.28363438314739253
Total used surplus: -0.06673863205965921
